# Orbital Mechanics Deep Dive

This notebook provides a comprehensive exploration of orbital mechanics concepts using Astrora.

**Topics covered:**
- Classical orbital elements (Keplerian elements)
- Conversions between representations
- Orbital energy and angular momentum
- Special orbit types (circular, elliptical, parabolic, hyperbolic)
- Orbital regimes (LEO, MEO, GEO, HEO)
- Kepler's laws
- Mean and eccentric anomaly

**Prerequisites:** Completion of 01_quickstart.ipynb or equivalent knowledge

## Setup

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astrora import Orbit, _core, bodies

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

print("✓ Imports successful!")

## 1. Classical Orbital Elements (COE)

The six classical orbital elements uniquely define an orbit:

1. **a** - Semi-major axis (size of orbit)
2. **e** - Eccentricity (shape of orbit)
3. **i** - Inclination (tilt relative to reference plane)
4. **Ω (RAAN)** - Right ascension of ascending node (orientation of orbital plane)
5. **ω (argp)** - Argument of periapsis (orientation within orbital plane)
6. **ν (nu)** - True anomaly (position along orbit)

Let's explore each element:

### Semi-major Axis (a) - Orbit Size

In [ ]:
# Create orbits with different semi-major axes
semi_major_axes = [6778, 12000, 26000, 42164]  # km
orbit_names = ['LEO', 'MEO', 'HEO', 'GEO']
colors = ['red', 'orange', 'yellow', 'blue']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Left plot: Orbits
earth_circle = plt.Circle((0, 0), 6371, color='lightblue', label='Earth')
ax1.add_patch(earth_circle)

for a, name, color in zip(semi_major_axes, orbit_names, colors):
    orbit = Orbit.from_classical(
        bodies.Earth,
        a=a * u.km,
        ecc=0.0 * u.one,
        inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
    )

    # Plot circular orbit
    theta = np.linspace(0, 2*np.pi, 100)
    x = a * np.cos(theta)
    y = a * np.sin(theta)
    ax1.plot(x, y, color=color, linewidth=2, label=f'{name} (a={a} km)')

ax1.set_xlabel('X (km)', fontsize=12)
ax1.set_ylabel('Y (km)', fontsize=12)
ax1.set_title('Effect of Semi-Major Axis', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_aspect('equal')
ax1.set_xlim(-50000, 50000)
ax1.set_ylim(-50000, 50000)

# Right plot: Orbital periods
periods = []
for a in semi_major_axes:
    orbit = Orbit.from_classical(
        bodies.Earth, a=a * u.km, ecc=0.0 * u.one,
        inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
    )
    periods.append(orbit.period.to_value('hour'))

ax2.bar(orbit_names, periods, color=colors, edgecolor='black', linewidth=2)
ax2.set_ylabel('Orbital Period (hours)', fontsize=12)
ax2.set_title('Kepler\'s Third Law: Period vs Semi-Major Axis', fontsize=14, fontweight='bold')
ax2.grid(True, axis='y', alpha=0.3)

# Add period values on bars
for i, (name, period) in enumerate(zip(orbit_names, periods)):
    ax2.text(i, period + 0.5, f'{period:.2f} hr', ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("Kepler's Third Law: T² ∝ a³")
print(f"  LEO (a={semi_major_axes[0]} km): T = {periods[0]:.2f} hours")
print(f"  GEO (a={semi_major_axes[3]} km): T = {periods[3]:.2f} hours ≈ 24 hours (geostationary)")

### Eccentricity (e) - Orbit Shape

In [ ]:
# Create orbits with different eccentricities
eccentricities = [0.0, 0.3, 0.6, 0.9]
ecc_labels = ['Circular (e=0)', 'Low (e=0.3)', 'Medium (e=0.6)', 'High (e=0.9)']
colors = ['blue', 'green', 'orange', 'red']

fig, ax = plt.subplots(figsize=(14, 10))

# Earth at origin
earth_circle = plt.Circle((0, 0), 6371, color='lightblue', label='Earth', zorder=10)
ax.add_patch(earth_circle)

a = 26000  # km (constant semi-major axis)

for ecc, label, color in zip(eccentricities, ecc_labels, colors):
    orbit = Orbit.from_classical(
        bodies.Earth,
        a=a * u.km,
        ecc=ecc * u.one,
        inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
    )

    # Calculate orbit positions
    theta = np.linspace(0, 2*np.pi, 200)
    positions = []
    for angle in theta:
        orb = Orbit.from_classical(
            bodies.Earth, a=a * u.km, ecc=ecc * u.one,
            inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=angle * u.rad
        )
        positions.append(orb.r.to_value('km'))
    positions = np.array(positions)

    ax.plot(positions[:, 0], positions[:, 1], color=color, linewidth=2, label=label)

    # Mark periapsis and apoapsis
    if ecc > 0:
        r_p = orbit.r_p.to_value('km')
        r_a = orbit.r_a.to_value('km')
        ax.scatter([r_p, -r_a], [0, 0], color=color, s=100, zorder=5)

ax.set_xlabel('X (km)', fontsize=12)
ax.set_ylabel('Y (km)', fontsize=12)
ax.set_title('Effect of Eccentricity (constant a=26,000 km)', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
ax.set_xlim(-60000, 30000)
ax.set_ylim(-45000, 45000)
plt.tight_layout()
plt.show()

print("Eccentricity Classification:")
print("  e = 0: Circular orbit")
print("  0 < e < 1: Elliptical orbit")
print("  e = 1: Parabolic trajectory (escape)")
print("  e > 1: Hyperbolic trajectory (escape)")

### Inclination (i) - Orbital Plane Tilt

In [ ]:
# Create orbits with different inclinations
inclinations = [0, 28.5, 51.6, 90, 98.2]  # degrees
inc_labels = ['Equatorial (0°)', 'Cape Canaveral (28.5°)', 'ISS (51.6°)',
              'Polar (90°)', 'Sun-Sync (98.2°)']
colors = ['blue', 'green', 'red', 'purple', 'orange']

fig = plt.figure(figsize=(14, 14))
ax = fig.add_subplot(111, projection='3d')

# Earth sphere
u_sphere = np.linspace(0, 2 * np.pi, 30)
v_sphere = np.linspace(0, np.pi, 30)
x_earth = 6371 * np.outer(np.cos(u_sphere), np.sin(v_sphere))
y_earth = 6371 * np.outer(np.sin(u_sphere), np.sin(v_sphere))
z_earth = 6371 * np.outer(np.ones(np.size(u_sphere)), np.cos(v_sphere))
ax.plot_surface(x_earth, y_earth, z_earth, color='lightblue', alpha=0.4)

# Equatorial plane
xx, yy = np.meshgrid(np.linspace(-10000, 10000, 10), np.linspace(-10000, 10000, 10))
zz = np.zeros_like(xx)
ax.plot_surface(xx, yy, zz, color='gray', alpha=0.1)

a = 8000  # km

for inc, label, color in zip(inclinations, inc_labels, colors):
    orbit = Orbit.from_classical(
        bodies.Earth,
        a=a * u.km,
        ecc=0.0 * u.one,
        inc=inc * u.deg,
        raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
    )

    # Propagate orbit
    theta = np.linspace(0, 2*np.pi, 100)
    positions = []
    for angle in theta:
        orb = Orbit.from_classical(
            bodies.Earth, a=a * u.km, ecc=0.0 * u.one,
            inc=inc * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=angle * u.rad
        )
        positions.append(orb.r.to_value('km'))
    positions = np.array(positions)

    ax.plot(positions[:, 0], positions[:, 1], positions[:, 2],
            color=color, linewidth=2, label=label)

ax.set_xlabel('X (km)', fontsize=10)
ax.set_ylabel('Y (km)', fontsize=10)
ax.set_zlabel('Z (km)', fontsize=10)
ax.set_title('Effect of Inclination', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=9)
ax.set_box_aspect([1,1,1])
plt.tight_layout()
plt.show()

print("Inclination Classification:")
print("  i = 0°: Equatorial (prograde)")
print("  0° < i < 90°: Prograde (same direction as Earth's rotation)")
print("  i = 90°: Polar")
print("  90° < i < 180°: Retrograde (opposite to Earth's rotation)")
print("  i ≈ 98°: Sun-synchronous (precesses with Sun)")

## 2. Orbital Energy and Angular Momentum

Two fundamental conserved quantities in orbital mechanics:

In [ ]:
# Create an elliptical orbit
orbit = Orbit.from_classical(
    bodies.Earth,
    a=20000 * u.km,
    ecc=0.5 * u.one,
    inc=30 * u.deg,
    raan=45 * u.deg,
    argp=60 * u.deg,
    nu=0 * u.deg
)

print("Orbital Properties:")
print(f"  Semi-major axis: {orbit.a.to(u.km):.2f}")
print(f"  Eccentricity: {orbit.ecc:.4f}")
print(f"  Inclination: {orbit.inc.to(u.deg):.2f}")
print(f"\nDerived Quantities:")
print(f"  Periapsis radius: {orbit.r_p.to(u.km):.2f}")
print(f"  Apoapsis radius: {orbit.r_a.to(u.km):.2f}")
print(f"  Period: {orbit.period.to(u.hour):.2f}")

# Calculate specific orbital energy
mu = bodies.Earth.mu.to_value('m^3/s^2')
a_m = orbit.a.to_value('m')
specific_energy = -mu / (2 * a_m)  # J/kg

print(f"\nEnergy:")
print(f"  Specific orbital energy: {specific_energy/1e6:.3f} MJ/kg")
print(f"  (Negative energy → bound orbit)")

# Calculate specific angular momentum magnitude
h_vec = np.cross(orbit.r.to_value('m'), orbit.v.to_value('m/s'))
h_mag = np.linalg.norm(h_vec)

print(f"\nAngular Momentum:")
print(f"  Specific angular momentum: {h_mag/1e9:.3f} × 10⁹ m²/s")
print(f"  Direction: perpendicular to orbital plane")

### Energy Conservation Along the Orbit

In [ ]:
# Propagate orbit and verify energy conservation
theta_array = np.linspace(0, 2*np.pi, 100)
energies = []
angular_momenta = []
radii = []

for nu in theta_array:
    orb = Orbit.from_classical(
        bodies.Earth,
        a=orbit.a, ecc=orbit.ecc, inc=orbit.inc,
        raan=orbit.raan, argp=orbit.argp, nu=nu * u.rad
    )

    r = orb.r.to_value('m')
    v = orb.v.to_value('m/s')
    r_mag = np.linalg.norm(r)
    v_mag = np.linalg.norm(v)

    # Specific energy: ε = v²/2 - μ/r
    energy = v_mag**2 / 2 - mu / r_mag
    energies.append(energy / 1e6)  # MJ/kg

    # Specific angular momentum
    h = np.cross(r, v)
    angular_momenta.append(np.linalg.norm(h) / 1e9)  # × 10⁹ m²/s

    radii.append(r_mag / 1000)  # km

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 12))

# Energy
ax1.plot(np.rad2deg(theta_array), energies, 'b-', linewidth=2)
ax1.axhline(y=np.mean(energies), color='r', linestyle='--',
            label=f'Mean: {np.mean(energies):.6f} MJ/kg')
ax1.set_ylabel('Specific Energy (MJ/kg)', fontsize=11)
ax1.set_title('Energy Conservation Along Orbit', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Angular momentum
ax2.plot(np.rad2deg(theta_array), angular_momenta, 'g-', linewidth=2)
ax2.axhline(y=np.mean(angular_momenta), color='r', linestyle='--',
            label=f'Mean: {np.mean(angular_momenta):.6f} × 10⁹ m²/s')
ax2.set_ylabel('Angular Momentum (× 10⁹ m²/s)', fontsize=11)
ax2.set_title('Angular Momentum Conservation', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend()

# Radius variation
ax3.plot(np.rad2deg(theta_array), radii, 'purple', linewidth=2)
ax3.axhline(y=orbit.r_p.to_value('km'), color='r', linestyle='--',
            label=f'Periapsis: {orbit.r_p.to_value("km"):.0f} km')
ax3.axhline(y=orbit.r_a.to_value('km'), color='b', linestyle='--',
            label=f'Apoapsis: {orbit.r_a.to_value("km"):.0f} km')
ax3.set_xlabel('True Anomaly (degrees)', fontsize=11)
ax3.set_ylabel('Radius (km)', fontsize=11)
ax3.set_title('Orbital Radius Variation', fontsize=13, fontweight='bold')
ax3.grid(True, alpha=0.3)
ax3.legend()

plt.tight_layout()
plt.show()

print(f"Energy standard deviation: {np.std(energies):.2e} MJ/kg (should be ~0)")
print(f"Angular momentum std dev: {np.std(angular_momenta):.2e} × 10⁹ m²/s (should be ~0)")

## 3. Anomalies: Position Along the Orbit

Three ways to specify position along an orbit:
- **True anomaly (ν)**: Geometric angle from periapsis
- **Eccentric anomaly (E)**: Auxiliary circle construction
- **Mean anomaly (M)**: Linearized time from periapsis

In [ ]:
# Create an elliptical orbit
ecc = 0.6
a = 15000  # km

orbit_anom = Orbit.from_classical(
    bodies.Earth,
    a=a * u.km,
    ecc=ecc * u.one,
    inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
)

# Calculate anomalies for various positions
true_anomalies = np.linspace(0, 2*np.pi, 100)
eccentric_anomalies = []
mean_anomalies = []

for nu in true_anomalies:
    # True to eccentric anomaly
    E = 2 * np.arctan(np.sqrt((1-ecc)/(1+ecc)) * np.tan(nu/2))
    eccentric_anomalies.append(E)

    # Eccentric to mean anomaly (Kepler's equation)
    M = E - ecc * np.sin(E)
    mean_anomalies.append(M)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Left: Anomaly relationships
ax1.plot(np.rad2deg(true_anomalies), np.rad2deg(eccentric_anomalies),
         'b-', linewidth=2, label='Eccentric Anomaly (E)')
ax1.plot(np.rad2deg(true_anomalies), np.rad2deg(mean_anomalies),
         'r-', linewidth=2, label='Mean Anomaly (M)')
ax1.plot(np.rad2deg(true_anomalies), np.rad2deg(true_anomalies),
         'k--', linewidth=1, alpha=0.5, label='True Anomaly (reference)')
ax1.set_xlabel('True Anomaly ν (degrees)', fontsize=12)
ax1.set_ylabel('Anomaly (degrees)', fontsize=12)
ax1.set_title(f'Anomaly Relationships (e={ecc})', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Right: Orbit with anomaly markers
positions = []
for nu in true_anomalies:
    orb = Orbit.from_classical(
        bodies.Earth, a=a * u.km, ecc=ecc * u.one,
        inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=nu * u.rad
    )
    positions.append(orb.r.to_value('km'))
positions = np.array(positions)

earth_circle = plt.Circle((0, 0), 6371, color='lightblue', label='Earth')
ax2.add_patch(earth_circle)
ax2.plot(positions[:, 0], positions[:, 1], 'b-', linewidth=2, label='Orbit')

# Mark special positions
special_nus = [0, np.pi/2, np.pi, 3*np.pi/2]
special_labels = ['Periapsis\n(ν=0°)', 'ν=90°', 'Apoapsis\n(ν=180°)', 'ν=270°']
for nu, label in zip(special_nus, special_labels):
    orb = Orbit.from_classical(
        bodies.Earth, a=a * u.km, ecc=ecc * u.one,
        inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=nu * u.rad
    )
    r = orb.r.to_value('km')
    ax2.scatter([r[0]], [r[1]], s=200, c='red', zorder=5)
    ax2.annotate(label, (r[0], r[1]), xytext=(10, 10),
                textcoords='offset points', fontsize=9, fontweight='bold')

ax2.set_xlabel('X (km)', fontsize=12)
ax2.set_ylabel('Y (km)', fontsize=12)
ax2.set_title('True Anomaly Positions', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.set_aspect('equal')

plt.tight_layout()
plt.show()

print("Kepler's Equation: M = E - e·sin(E)")
print("  Mean anomaly (M) increases linearly with time")
print("  Eccentric anomaly (E) is intermediate geometric construction")
print("  True anomaly (ν) is actual angular position from periapsis")

## 4. Special Orbit Types

### Escape Trajectories

In [ ]:
# Create different orbit types based on eccentricity
orbit_types = [
    {'ecc': 0.0, 'name': 'Circular', 'color': 'blue'},
    {'ecc': 0.5, 'name': 'Elliptical', 'color': 'green'},
    {'ecc': 0.95, 'name': 'Highly Elliptical', 'color': 'orange'},
]

fig, ax = plt.subplots(figsize=(14, 10))

earth_circle = plt.Circle((0, 0), 6371, color='lightblue', label='Earth', zorder=10)
ax.add_patch(earth_circle)

a = 20000  # km

for orbit_type in orbit_types:
    ecc = orbit_type['ecc']
    name = orbit_type['name']
    color = orbit_type['color']

    orbit = Orbit.from_classical(
        bodies.Earth, a=a * u.km, ecc=ecc * u.one,
        inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=0 * u.deg
    )

    # Calculate specific energy
    mu = bodies.Earth.mu.to_value('m^3/s^2')
    energy = -mu / (2 * orbit.a.to_value('m')) / 1e6  # MJ/kg

    theta = np.linspace(0, 2*np.pi, 200)
    positions = []
    for nu in theta:
        orb = Orbit.from_classical(
            bodies.Earth, a=a * u.km, ecc=ecc * u.one,
            inc=0 * u.deg, raan=0 * u.deg, argp=0 * u.deg, nu=nu * u.rad
        )
        positions.append(orb.r.to_value('km'))
    positions = np.array(positions)

    ax.plot(positions[:, 0], positions[:, 1], color=color, linewidth=2.5,
            label=f'{name} (e={ecc}, E={energy:.2f} MJ/kg)')

ax.set_xlabel('X (km)', fontsize=12)
ax.set_ylabel('Y (km)', fontsize=12)
ax.set_title('Orbit Classification by Eccentricity', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')
ax.set_xlim(-50000, 25000)
ax.set_ylim(-37500, 37500)
plt.tight_layout()
plt.show()

print("Energy and Orbit Type:")
print("  E < 0: Bound orbit (elliptical, including circular)")
print("  E = 0: Parabolic escape trajectory")
print("  E > 0: Hyperbolic escape trajectory")

## 5. State Vector Conversions

Convert between Cartesian state vectors (r, v) and classical orbital elements (COE):

In [ ]:
# Start with state vectors
r_initial = np.array([7000e3, 2000e3, 1000e3])  # m
v_initial = np.array([1000.0, 7000.0, 500.0])   # m/s

print("Initial State Vectors:")
print(f"  Position: {r_initial/1000} km")
print(f"  Velocity: {v_initial} m/s")

# Convert to classical orbital elements using Rust backend
mu = bodies.Earth.mu.to_value('m^3/s^2')
coe = _core.rv_to_coe(r_initial, v_initial, mu)

print(f"\nClassical Orbital Elements:")
print(f"  Semi-major axis (a): {coe[0]/1000:.2f} km")
print(f"  Eccentricity (e): {coe[1]:.6f}")
print(f"  Inclination (i): {np.rad2deg(coe[2]):.4f}°")
print(f"  RAAN (Ω): {np.rad2deg(coe[3]):.4f}°")
print(f"  Argument of periapsis (ω): {np.rad2deg(coe[4]):.4f}°")
print(f"  True anomaly (ν): {np.rad2deg(coe[5]):.4f}°")

# Convert back to state vectors
r_final, v_final = _core.coe_to_rv(
    coe[0], coe[1], coe[2], coe[3], coe[4], coe[5], mu
)

print(f"\nConverted Back to State Vectors:")
print(f"  Position: {r_final/1000} km")
print(f"  Velocity: {v_final} m/s")

# Verify roundtrip accuracy
r_error = np.linalg.norm(r_final - r_initial)
v_error = np.linalg.norm(v_final - v_initial)

print(f"\nRoundtrip Error:")
print(f"  Position error: {r_error:.2e} m")
print(f"  Velocity error: {v_error:.2e} m/s")
print(f"  ✓ High-precision conversions powered by Rust!")

## Summary

In this notebook, you learned:

✅ The six classical orbital elements and their physical meaning

✅ How each element affects the orbit shape and orientation

✅ Conservation of orbital energy and angular momentum

✅ The relationship between true, eccentric, and mean anomaly

✅ Classification of orbits by eccentricity

✅ High-precision conversions between state vectors and orbital elements

### Next Steps

Continue with:
- **03_maneuvers_transfers.ipynb** - Orbital maneuvers and transfers
- **04_visualization_plotting.ipynb** - Advanced visualization techniques
- **05_advanced_techniques.ipynb** - Monte Carlo, perturbations, mission analysis